In [5]:
import api_files.df_functions as df_func 
import api_files.calculations as calc
import pandas as pd
from train_test_divide import extract_spammers_scrapers, extract_normal, split_train_test

Hur kommer ett request se ut när det kommer in?
- sträng i Redis? 
- sparas i sorted sets?
- hämta som mest ett fixt antal, ex 50.

Efter allt är hämtat från Redis:
- spara undan i en data frame med varje värde i rätt kolumn.

Dataframe:

userID, sessionID, URL, osv.        time mellan req         unika sessionID
user1   session1, login
user1   session1  chat
user1   session1  sendMessage

Kör in data framen i en funktion som gör nedanstående steg med hjälp av diverse helper-metoder:
För user1: 
- vad är den genomsnittliga tiden mellan requests?
- tokens under 5 min (skapa en funktion som först grupperar och sen räknar ut medelvärde av antal sessionIDs för alla 5-min intervall)
När dessa två steg är klara, behöver vi inte längre ta hänsyn till timestamps eller sessionID. Resultaten från dessa beräkningar 
sparas undan i egna variabler som läggs till i slutskedet av konstruktionen av vår test-dataframe.

- vektorisera URLer och ta fram:
    - varians, 
    - unika requests,
    - längsta följd av upprepade förfrågningar

In [7]:
"""Code for fetching the data"""

#Fetch data from csv file
data = df_func.read_csv_file('csv_files/requests.csv')

#Extracts a list of all unique users in dataframe
users = df_func.extract_users(data)
vect = calc.create_vectorizer(data)

In [8]:
df_chunks = df_func.split_user_df(data, users[0]) 
df_chunks[1].to_csv('csv_files/datachunk.csv', index=False)
df_chunks[0]
#clean_reqlogs(df_chunks[0])['request_logs'].to_list()

,timestamp,userID,sessionID,expiring,URL
0,1676373343936,user-1003,c85cbd00-d2fc-4cda-9f5c-66616847d851,1676373495816,/chat/send/442
2,1676373344456,user-1003,c85cbd00-d2fc-4cda-9f5c-66616847d851,1676373495816,/chat/send/969
26,1676373348109,user-1003,c85cbd00-d2fc-4cda-9f5c-66616847d851,1676373495816,/searchUsers?page=2036
29,1676373348626,user-1003,c85cbd00-d2fc-4cda-9f5c-66616847d851,1676373495816,/chat/send/647
31,1676373349146,user-1003,c85cbd00-d2fc-4cda-9f5c-66616847d851,1676373495816,/chat/send/886
32,1676373349666,user-1003,c85cbd00-d2fc-4cda-9f5c-66616847d851,1676373495816,/chat/send/885
35,1676373350186,user-1003,c85cbd00-d2fc-4cda-9f5c-66616847d851,1676373495816,/chat/send/866
38,1676373350706,user-1003,c85cbd00-d2fc-4cda-9f5c-66616847d851,1676373495816,/chat/send/888
44,1676373351226,user-1003,c85cbd00-d2fc-4cda-9f5c-66616847d851,1676373495816,/chat/send/267
47,1676373351746,user-1003,c85cbd00-d2fc-4cda-9f5c-66616847d851,1676373495816,/chat/send/709


In [4]:
"""
Code for creating csv-file containing information about average request frequency, average number of tokens per 5 mins, 
longest consecutive chain of repetitive requests for every chunk of 50 requests for each user
"""

df_calculations = pd.DataFrame()
for user in users:
    data_chunks = df_func.split_user_df(data, user)
    avg_req_frequences = []
    avg_tokens = []
    repetitions = []
    var_scores = []
    sequence_time = []

    for i in range(0, len(data_chunks)):
        avg_req_frequences.append(calc.calc_avg_timediff(data_chunks[i]))
        avg_tokens.append(calc.avg_tokens_5mins(data_chunks[i]))
        repetitions.append(calc.longest_consec(data_chunks[i]))
        var_scores.append(calc.get_variance_score(data_chunks[i], vect))
        sequence_time.append(calc.sequence_time_length(data_chunks[i]))
    
    user_df = pd.DataFrame()
    user_df['request_freq'] = avg_req_frequences
    user_df['avg_tokens'] = avg_tokens
    user_df['longest_consec'] = repetitions
    user_df['var_score'] = var_scores
    user_df['sequence_time'] = sequence_time
    user_df['user'] = user
    df_calculations = pd.concat([df_calculations, user_df])

df_calculations.reset_index(inplace=True)
df_calculations = df_calculations.drop(columns='index')
df_calculations.to_csv('csv_files/calculations.csv', index=False) 

In [5]:
data_calc = pd.read_csv('csv_files/calculations.csv')
data_calc

,request_freq,avg_tokens,longest_consec,var_score,sequence_time,user
0,2.101296,2.000000,15.0,203.095238,113.470,user-1003
1,2.696333,3.000000,9.0,170.066667,145.602,user-1003
2,2.108796,2.000000,15.0,179.095238,113.875,user-1003
3,2.227037,4.000000,9.0,174.923810,120.260,user-1003
4,2.011852,2.000000,16.0,210.523810,108.640,user-1003
...,...,...,...,...,...,...
5270,598.277932,0.147727,2.0,31.238095,26324.229,user-147
5271,387.771702,0.096774,9.0,27.285714,18225.270,user-754
5272,642.863652,0.111111,5.0,24.352381,29571.728,user-754
5273,898.434058,0.121019,3.0,32.495238,46718.571,user-641


In [6]:
df_spam = extract_spammers_scrapers(data_calc)
split_train_test(extract_normal(data_calc, df_spam), df_spam)